In [1]:
// Imports
import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection 
import org.apache.spark.sql.functions._

//import scala.sqlContext.implicits._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.util.SizeEstimator


import org.apache.spark.input.PortableDataStream
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import com.amazonaws.auth.BasicAWSCredentials

//import org.apache.spark.sql.cassandra._

//import com.datastax.spark.connector._
import org.apache.spark.sql.types.IntegerType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1611248756501_0001,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.util.SizeEstimator
import org.apache.spark.input.PortableDataStream
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import com.amazonaws.auth.BasicAWSCredentials
import org.apache.spark.sql.types.IntegerType


In [2]:
def fileDownloader(urlOfFileToDownload: String, fileName: String) = {
    val url = new URL(urlOfFileToDownload)
    val connection = url.openConnection().asInstanceOf[HttpURLConnection]
    connection.setConnectTimeout(5000)
    connection.setReadTimeout(5000)
    connection.connect()

    if (connection.getResponseCode >= 400)
        println("error")
    else
        url #> new File(fileName) !!
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fileDownloader: (urlOfFileToDownload: String, fileName: String)Any


In [3]:
fileDownloader("http://data.gdeltproject.org/gdeltv2/masterfilelist.txt", "/tmp/masterfilelist.txt") // save the list file to the Spark Master
fileDownloader("http://data.gdeltproject.org/gdeltv2/masterfilelist-translation.txt", "/tmp/masterfilelist_translation.txt") //same for Translation file

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res10: Any = ""
res11: Any = ""


# Telechargement massive pour Master

In [4]:
import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}
@transient val s3Client: AmazonS3 = AmazonS3ClientBuilder.defaultClient()
s3Client.putObject("testfuret", "masterfilelist.txt", new File("/tmp/masterfilelist.txt") )
s3Client.putObject("testfuret", "masterfilelist_translation.txt", new File( "/tmp/masterfilelist_translation.txt")) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}
s3Client: com.amazonaws.services.s3.AmazonS3 = com.amazonaws.services.s3.AmazonS3Client@4d4365ba
res12: com.amazonaws.services.s3.model.PutObjectResult = com.amazonaws.services.s3.model.PutObjectResult@614bddcb
res13: com.amazonaws.services.s3.model.PutObjectResult = com.amazonaws.services.s3.model.PutObjectResult@6bc3ff93


In [5]:
// English Data
val list_csv = spark.read.format("csv").option("delimiter", " ").
    csv("s3://testfuret/masterfilelist.txt").
    withColumnRenamed("_c0","size").
    withColumnRenamed("_c1","hash").
    withColumnRenamed("_c2","url")

//val list_2020_tot = list_csv.where(col("url").like("%/2020%")) 
val list_2020_tot = list_csv.where(col("url").like("%gkg%")) 
list_2020_tot.show(10,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list_csv: org.apache.spark.sql.DataFrame = [size: string, hash: string ... 1 more field]
list_2020_tot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [size: string, hash: string ... 1 more field]
+--------+--------------------------------+---------------------------------------------------------------+
|size    |hash                            |url                                                            |
+--------+--------------------------------+---------------------------------------------------------------+
|10768507|ea8dde0beb0ba98810a92db068c0ce99|http://data.gdeltproject.org/gdeltv2/20150218230000.gkg.csv.zip|
|10269336|2f1a504a3c4558694ade0442e9a5ae6f|http://data.gdeltproject.org/gdeltv2/20150218231500.gkg.csv.zip|
|11279827|66b03e2efd7d51dabf916b1666910053|http://data.gdeltproject.org/gdeltv2/20150218233000.gkg.csv.zip|
|11212939|cd20f295649b214dd16666ca451b9994|http://data.gdeltproject.org/gdeltv2/20150218234500.gkg.csv.zip|
|9728953 |8f4b26e134bd6605cce2d32e92e

In [6]:
def StokeFileS3(URL: String) = {
    val fileName = URL.split("/").last
    val dir = "/mnt/tmp/"
    val localFileName = dir + fileName
    try {
            fileDownloader(URL,  localFileName)
            @transient val s3Client: AmazonS3 = AmazonS3ClientBuilder.defaultClient()
            val localFile = new File(localFileName)
            s3Client.putObject("testfuret/Master_file", fileName, localFile)   
            localFile.delete()
        
    } catch {
        case e: java.io.FileNotFoundException => 
        case e: com.amazonaws.SdkClientException => 

    }    
    
    
}



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StokeFileS3: (URL: String)AnyVal


# Lancement du du telechargement massive

In [ ]:
// massive
list_2020_tot.select("url").repartition(100).foreach( r=> {
    val URL = r.getAs[String](0)
    StokeFileS3(URL)
})
//15h52  -40 min de telechargement




In [13]:
var URL = "http://data.gdeltproject.org/gdeltv2/20201214121500.mentions.CSV.zip"
StokeFileS3(URL)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

URL: String = http://data.gdeltproject.org/gdeltv2/20201214121500.mentions.CSV.zip
error
res25: AnyVal = ()


In [14]:
var URL1 =  "http://data.gdeltproject.org/gdeltv2/20200101000000.export.CSV.zip" 
StokeFileS3(URL1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

URL1: String = http://data.gdeltproject.org/gdeltv2/20200101000000.export.CSV.zip
res26: AnyVal = true


# Telechargement massive pour Master Translat

In [7]:
// English Data
val list_csv_translate = spark.read.format("csv").option("delimiter", " ").
    csv("s3://testfuret/masterfilelist_translation.txt").   
    withColumnRenamed("_c0","size").
    withColumnRenamed("_c1","hash").
    withColumnRenamed("_c2","url")


val list_2020_translate_tot = list_csv_translate.where(col("url").like("%/2020%")) 
list_2020_translate_tot.show(10,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list_csv_translate: org.apache.spark.sql.DataFrame = [size: string, hash: string ... 1 more field]
list_2020_translate_tot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [size: string, hash: string ... 1 more field]
+-------+--------------------------------+--------------------------------------------------------------------------------+
|size   |hash                            |url                                                                             |
+-------+--------------------------------+--------------------------------------------------------------------------------+
|54676  |832b70ff3b1656dbce66bf671fdf7c09|http://data.gdeltproject.org/gdeltv2/20200101000000.translation.export.CSV.zip  |
|43214  |4826b428cf5a7d11d6c2feb98b1e7ea9|http://data.gdeltproject.org/gdeltv2/20200101000000.translation.mentions.CSV.zip|
|5505853|9a7dcb3feda6884fbb1c1f4cd15b2a31|http://data.gdeltproject.org/gdeltv2/20200101000000.translation.gkg.csv.zip     |
|35583  |028d8fde74abf76afe41

In [13]:
def StokeFileS3(URL: String) = {
    val fileName = URL.split("/").last
    val dir = "/mnt/tmp/"
    val localFileName = dir + fileName
    try {
            fileDownloader(URL,  localFileName)
            @transient val s3Client: AmazonS3 = AmazonS3ClientBuilder.defaultClient()
            val localFile = new File(localFileName)
            s3Client.putObject("testfuret/Master_Translate_file ", fileName, localFile)     
            localFile.delete()
        
    } catch {
        case e: java.io.FileNotFoundException => 
        case e: com.amazonaws.SdkClientException => 

    }    
    
    
}



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StokeFileS3: (URL: String)AnyVal


In [17]:
list_2020_translate_tot.select("url").repartition(100).foreach( r=> {
    val URL = r.getAs[String](0)
    StokeFileS3(URL)
})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
var URL2="http://data.gdeltproject.org/gdeltv2/20200101000000.translation.export.CSV.zip"
StokeFileS3(URL2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

URL2: String = http://data.gdeltproject.org/gdeltv2/20200101000000.translation.export.CSV.zip
res37: AnyVal = true
